In [1]:
import os
import sys
import torch
import cornac

from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.evaluation.python_evaluation import (
    map,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
)
from recommenders.utils.notebook_utils import store_metadata

print(f"System version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"Cornac version: {cornac.__version__}")

/home/sokolov/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


System version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
PyTorch version: 2.5.1+cu124
Cornac version: 2.2.2


In [2]:

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

# top k items to recommend
TOP_K = 10

# Model parameters
LATENT_DIM = 50
ENCODER_DIMS = [100]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
NUM_EPOCHS = 500
BATCH_SIZE = 128
LEARNING_RATE = 0.001

In [3]:
from utils import evaluate, load_data

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

##### read data

In [5]:
user_item_data, user_meta_data, item_meta_data, test_pairs_data = load_data()

In [6]:
# np.uint8 -> np.int16 cast to allow subtraction
user_item_data[user_item_data.dtypes[user_item_data.dtypes==np.uint8].index] = \
    user_item_data[user_item_data.dtypes[user_item_data.dtypes==np.uint8].index].astype(np.int16)
# single column for likes and dislikes
user_item_data["explicit"] = user_item_data.like - user_item_data.dislike

In [7]:
user_item_data = user_item_data[["user_id", "item_id", "explicit"]]

In [8]:
ui_train, ui_val = train_test_split(user_item_data,
                                    test_size=0.15,
                                    random_state=42,
                                    shuffle=False)

In [9]:
train_set = cornac.data.Dataset.from_uir(ui_train.values,seed=SEED,)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 183404
Number of items: 337707


In [5]:
!python -m pip install distutils

ERROR: Could not find a version that satisfies the requirement distutils (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for distutils


In [3]:
!python -m pip install numpy==1.9 scipy==0.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 8.0 MB/s eta 0:00:00ta 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached scipy-0.13.0.zip (11.3 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [17 lines of output]
      Running from numpy source directory.
      /tmp/pip-install-5jtah43c/numpy_2d538da5c2254aa78c47c3fc0769a0f3/numpy/distutils/misc_util.py:415: SyntaxWarning: "is" with a literal. Did you mean "=="?
        return is_string(s) and ('*' in s or '?' is s)
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-5jtah43c/numpy_2d538da5c2254aa78c47c3fc0769a0f3/setup.py", line 251, in <module>
          setup_package()
        File "/tmp/pip-install-5jtah43c/numpy_2d538da5c2254aa78c47c3fc0769a0f3/setup.py", line 235, in

In [5]:
scipy.__version__

AttributeError: module 'scipy' has no attribute '__version__'

In [ ]:
!

In [10]:
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/500 [00:11<?, ?it/s]


AttributeError: 'csc_matrix' object has no attribute 'A'